<a href="https://colab.research.google.com/github/leeje008/CUAI_NLP_STUDY/blob/master/optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time



import warnings
warnings.filterwarnings(action='ignore')

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier


import lightgbm as lgb
from lightgbm import LGBMClassifier


from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from imblearn.over_sampling import SMOTE
from sklearn.metrics import roc_auc_score, make_scorer

In [5]:
df = pd.read_csv('/content/drive/MyDrive/pd_speech_features.csv')
df.head()

,id,gender,PPE,DFA,RPDE,numPulses,numPeriodsPulses,meanPeriodPulses,stdDevPeriodPulses,locPctJitter,locAbsJitter,rapJitter,ppq5Jitter,ddpJitter,locShimmer,locDbShimmer,apq3Shimmer,apq5Shimmer,apq11Shimmer,ddaShimmer,meanAutoCorrHarmonicity,meanNoiseToHarmHarmonicity,meanHarmToNoiseHarmonicity,minIntensity,maxIntensity,meanIntensity,f1,f2,f3,f4,b1,b2,b3,b4,GQ_prc5_95,GQ_std_cycle_open,GQ_std_cycle_closed,GNE_mean,GNE_std,GNE_SNR_TKEO,...,tqwt_skewnessValue_dec_34,tqwt_skewnessValue_dec_35,tqwt_skewnessValue_dec_36,tqwt_kurtosisValue_dec_1,tqwt_kurtosisValue_dec_2,tqwt_kurtosisValue_dec_3,tqwt_kurtosisValue_dec_4,tqwt_kurtosisValue_dec_5,tqwt_kurtosisValue_dec_6,tqwt_kurtosisValue_dec_7,tqwt_kurtosisValue_dec_8,tqwt_kurtosisValue_dec_9,tqwt_kurtosisValue_dec_10,tqwt_kurtosisValue_dec_11,tqwt_kurtosisValue_dec_12,tqwt_kurtosisValue_dec_13,tqwt_kurtosisValue_dec_14,tqwt_kurtosisValue_dec_15,tqwt_kurtosisValue_dec_16,tqwt_kurtosisValue_dec_17,tqwt_kurtosisValue_dec_18,tqwt_kurtosisValue_dec_19,tqwt_kurtosisValue_dec_20,tqwt_kurtosisValue_dec_21,tqwt_kurtosisValue_dec_22,tqwt_kurtosisValue_dec_23,tqwt_kurtosisValue_dec_24,tqwt_kurtosisValue_dec_25,tqwt_kurtosisValue_dec_26,tqwt_kurtosisValue_dec_27,tqwt_kurtosisValue_dec_28,tqwt_kurtosisValue_dec_29,tqwt_kurtosisValue_dec_30,tqwt_kurtosisValue_dec_31,tqwt_kurtosisValue_dec_32,tqwt_kurtosisValue_dec_33,tqwt_kurtosisValue_dec_34,tqwt_kurtosisValue_dec_35,tqwt_kurtosisValue_dec_36,class
0,0,1,0.85247,0.71826,0.57227,240,239,0.008064,0.000087,0.00218,0.000018,0.00067,0.00129,0.00200,0.05883,0.517,0.03011,0.03496,0.04828,0.09034,0.970805,0.036223,18.995,69.997496,76.088046,72.465512,539.342735,1031.849040,2447.162183,3655.054806,101.092218,83.147440,255.214830,396.643631,0.77778,11.7245,2.8277,1.17300,0.26512,0.083127,...,0.071728,0.010352,-2.73030,66.5007,36.9934,26.3508,51.9577,21.5451,8.2488,8.0024,6.9635,6.9189,5.0622,4.6068,7.7218,2.7224,2.4171,2.9383,4.2077,3.1541,2.8531,2.7496,2.1550,2.9457,2.1993,1.9830,1.8314,2.0062,1.6058,1.5466,1.5620,2.6445,3.8686,4.2105,5.1221,4.4625,2.6202,3.0004,18.9405,1
1,0,1,0.76686,0.69481,0.53966,234,233,0.008258,0.000073,0.00195,0.000016,0.00052,0.00112,0.00157,0.05516,0.502,0.02320,0.03675,0.06195,0.06961,0.984322,0.017974,21.497,67.415903,73.046374,71.528945,564.363614,1016.367294,2383.565201,3498.681572,58.465428,86.487292,248.357127,218.229722,0.81250,13.8284,2.8908,1.02210,0.22004,0.127410,...,0.729330,0.780410,5.22940,8643.9860,3962.0554,2976.2411,4329.0607,4005.1329,1127.6762,116.5331,20.2332,13.6395,12.5514,13.6965,60.8974,6.2958,3.0323,3.4039,4.3794,2.4329,2.0585,2.1839,2.2061,3.0999,1.9824,1.6227,1.5783,2.0470,1.5772,1.5530,1.5589,3.6107,23.5155,14.1962,11.0261,9.5082,6.5245,6.3431,45.1780,1
2,0,1,0.85083,0.67604,0.58982,232,231,0.008340,0.000060,0.00176,0.000015,0.00057,0.00111,0.00171,0.09902,0.897,0.05094,0.06497,0.07772,0.15282,0.974846,0.026313,17.651,62.661706,71.633549,68.086583,548.444604,1032.406341,2357.826954,3678.128717,160.387771,54.685168,151.694847,84.240339,0.81818,26.9273,2.6975,0.84951,0.15756,0.116890,...,0.269510,-0.005522,0.35054,29.2717,32.4971,38.9453,85.1480,62.5132,9.7308,6.8890,6.1357,5.4926,3.9078,4.0864,6.9828,3.4411,3.5173,3.7204,4.2212,3.4881,3.4851,3.3007,2.0427,3.1436,2.1203,1.6627,1.6731,3.2597,1.5921,1.5399,1.5643,2.3308,9.4959,10.7458,11.0177,4.8066,2.9199,3.1495,4.7666,1
3,1,0,0.41121,0.79672,0.59257,178,177,0.010858,0.000183,0.00419,0.000046,0.00149,0.00268,0.00446,0.05451,0.527,0.02395,0.02857,0.04462,0.07185,0.968343,0.042003,19.865,76.306989,81.000749,79.190593,819.529588,1201.813897,3154.035654,4122.163933,238.667052,191.984916,573.752909,526.147599,0.98548,139.5744,1.6961,0.83405,0.17295,0.147370,...,0.366920,-0.492650,0.19164,591.0116,65.3827,53.9852,45.4458,34.6650,88.3259,46.6869,23.2911,19.2022,42.3957,100.3649,7.1967,6.3892,6.5496,6.0264,4.7656,4.8909,4.2531,3.0295,2.0362,1.8478,2.5776,2.2064,1.9491,1.9120,1.8829,6.9761,3.7805,3.5664,5.2558,14.0403,4.2235,4.6857,4.8460,6.2650,4.0603,1
4,1,0,0.3

In [6]:
df.iloc[:,-1].value_counts() # imbalaced data

1    564
0    192
Name: class, dtype: int64

In [7]:
def get_clf_eval(y_test,pred):
    confusion = confusion_matrix(y_test, pred)
    accuracy = accuracy_score(y_test, pred)
    precision = precision_score(y_test, pred)
    recall = recall_score(y_test, pred)
    roc = roc_auc_score(y_test, pred)
    g_mean = geometric_mean_score(y_test, pred)
    print('오차행렬')
    print(confusion)
    print('정확도: {0:4f}, 정밀도 {1:4f}, 재현율 {2:4f}, AUC {3:4f},G_MEAN {4:4f}'.format(accuracy , precision , recall,roc,g_mean))
    

def get_model_train_eval(model,ftr_train = None, ftr_test = None, tgt_train = None, tgt_test = None):
    model.fit(ftr_train, tgt_train)
    pred = model.predict(ftr_test)
    get_clf_eval(tgt_test, pred)



import warnings
warnings.filterwarnings(action='ignore')

In [8]:
y = df['class']
x = df.drop('class',axis = 1)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3,
                                                    stratify = y ,random_state=42)

In [9]:
scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

smote = SMOTE(random_state = 101)

x_resampled, y_resampled = smote.fit_resample(x_train,y_train)


seed = 101

# Grid-search-LR

In [8]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", LogisticRegression())
                            ])

param_grid = {"clf__max_iter": [1000,2000,3000,4000,5000],
              'clf__C' : [0, 0.5, 1, 1.5, 2]
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))


Fitting 10 folds for each of 25 candidates, totalling 250 fits
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.2s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=1000; total time=   0.1s
[CV] END .......................clf__C=0, clf__max_iter=2000; total time=   0.1s
[CV] END .......................clf__C=0, clf_

# Random-LR

In [10]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform

start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", LogisticRegression())
                            ])

param_grid = {"clf__max_iter": [1000, 2000, 3000, 4000, 5000],
              'clf__C' : uniform(0,2)
              }



gs_pipeline = RandomizedSearchCV(pipeline,param_distributions=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.4s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.2s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.1s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.1s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.2s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.2s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.2s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.2s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.2s
[CV] END .......clf__C=0.372256972829216, clf__max_iter=5000; total time=   0.1s
[CV] END ......clf__C=1.9293426211924736, clf__max_iter=1000; total time=   0.1s
[CV] END ......clf__C=1.9293426211924736, clf_

# bayesian LR

In [11]:
!pip install bayesian-optimization
!pip install hyperopt

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-py3-none-any.whl size=11685 sha256=594cccbcbde492e1c81af9f3a68eb7acbea0d10331dcd894591136992e9e4991
  Stored in directory: /root/.cache/pip/wheels/fd/9b/71/f127d694e02eb40bcf18c7ae9613b88a6be4470f57a8528c5b
Successfully built bayesian-optimization


In [12]:
pca = PCA(n_components=  23)
pca.fit(x_resampled)
ba_tr = pca.fit_transform(x_resampled)
ba_te = pca.transform(x_test)

In [ ]:
"""
seed = 101

import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from scipy.stats import uniform
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization


start = time.time()


space={'max_iter': hp.quniform("max_iter", 1000, 5000, 1000),
       "C" : hp.quniform('C', 0, 2, 0.1)}

# 목적 함수 정의
# n_estimators, max_depth와 같은 반드시 int 타입을 가져야 하는 hyperparamter는 int로 타입 캐스팅 합니다.
def hyperparameter_tuning(space):
  model=LogisticRegression(max_iter = int(space['max_iter']),
                           C = space['C'])
  
  scores = cross_val_score(model, ba_tr, y_resampled, scoring=make_scorer(roc_auc_score), cv=10).mean()
    

    # 평가 방식 선정
  return {'loss': 1 - scores, 'status': STATUS_OK, 'model': model}



trials = Trials()

# best에 최적의 하이퍼 파라미터를 return 받습니다.
best = fmin(fn=hyperparameter_tuning,
            space=space,
            algo=tpe.suggest,
            max_evals=50, # 최대 반복 횟수를 지정합니다.
            trials=trials)

# 최적화된 결과를 int로 변환해야하는 파라미터는 타입 변환을 수행합니다.
best['max_iter'] = int(best['max_iter'])
best['C'] = best['C']

print(best)

lr = LogisticRegression(**best)
lr.fit(ba_tr, y_resampled)
pred = lr.predict(ba_te)
print(roc_auc_score(y_test, pred))
#


print('It takes %s minutes' % ((time.time() - start)/60))

"""



In [13]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

def lr_bo_cl(max_iter, C):
    params_lr = {}
    params_lr['max_iter'] = round(max_iter)
    params_lr['C'] = C
    scores = cross_val_score( LogisticRegression(random_state=123, **params_lr),
                             ba_tr, y_resampled, scoring=make_scorer(roc_auc_score), cv=10).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_lr ={
    'max_iter':(1000, 5000),
    'C':(0, 2)}
lr_bo = BayesianOptimization(lr_bo_cl, params_lr, random_state=111)
lr_bo.maximize(init_points=20, n_iter=50)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   |     C     | max_iter  |
-------------------------------------------------
|  1        |  0.8086   |  1.224    |  1.676e+0 |
|  2        |  0.8098   |  0.8721   |  4.077e+0 |
|  3        |  0.8098   |  0.5907   |  1.597e+0 |
|  4        |  0.8137   |  0.04496  |  2.681e+0 |
|  5        |  0.8111   |  0.4774   |  2.351e+0 |
|  6        |  0.8086   |  1.981    |  1.951e+0 |
|  7        |  0.8124   |  0.1624   |  3.678e+0 |
|  8        |  0.8086   |  1.242    |  2.097e+0 |
|  9        |  0.8086   |  0.9324   |  1.473e+0 |
|  10       |  0.8124   |  0.1479   |  4.603e+0 |
|  11       |  0.8086   |  1.588    |  4.362e+0 |
|  12       |  0.8086   |  1.63     |  4.964e+0 |
|  13       |  0.8086   |  1.155    |  4.255e+0 |
|  14       |  0.8098   |  0.8426   |  1.11e+03 |
|  15       |  0.8086   |  0.9083   |  1.421e+0 |
|  16       |  0.8086   |  1.634    |  3.791e+0 |
|  17       |  0.8086   |  1.131    |  2.097e+0 |
|  18       |  0.8086   |  1.997    |  1.552e+0 |


In [15]:
best = lr_bo.max['params']
best['max_iter'] = round(best['max_iter'])


lr = LogisticRegression(**best)
lr.fit(ba_tr, y_resampled)
pred = lr.predict(ba_te)
print(roc_auc_score(y_test, pred))

print(best)

0.7065394817384207
{'C': 0.0627310080311847, 'max_iter': 4603}


# KNN

In [16]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", KNeighborsClassifier())
                            ])

param_grid = {"clf__n_neighbors": list(range(1,20,1))
              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))


Fitting 10 folds for each of 19 candidates, totalling 190 fits
[CV] END .................................clf__n_neighbors=1; total time=   0.1s
[CV] END .................................clf__n_neighbors=1; total time=   0.2s
[CV] END .................................clf__n_neighbors=1; total time=   0.2s
[CV] END .................................clf__n_neighbors=1; total time=   0.2s
[CV] END .................................clf__n_neighbors=1; total time=   0.3s
[CV] END .................................clf__n_neighbors=1; total time=   0.1s
[CV] END .................................clf__n_neighbors=1; total time=   0.3s
[CV] END .................................clf__n_neighbors=1; total time=   0.3s
[CV] END .................................clf__n_neighbors=1; total time=   0.2s
[CV] END .................................clf__n_neighbors=1; total time=   0.2s
[CV] END .................................clf__n_neighbors=2; total time=   0.3s
[CV] END .................................clf_

In [18]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", KNeighborsClassifier())
                            ])

param_grid = {"clf__n_neighbors": list(range(1,20,1))
              }



gs_pipeline = RandomizedSearchCV(pipeline,param_distributions=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END ................................clf__n_neighbors=12; total time=   0.2s
[CV] END .................................clf__n_neighbors=2; total time=   0.2s
[CV] END .................................clf_

In [22]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

def knn_bo_cl(n_neighbors):
    params_knn = {}
    params_knn['n_neighbors'] = round(n_neighbors)
    scores = cross_val_score( KNeighborsClassifier( **params_knn),
                             ba_tr, y_resampled, scoring=make_scorer(roc_auc_score), cv=10).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_knn ={
    'n_neighbors':(1, 21)
 }
knn_bo = BayesianOptimization(knn_bo_cl, params_knn, random_state=111)
knn_bo.maximize(init_points=20, n_iter=50)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | n_neig... |
-------------------------------------
|  1        |  0.8442   |  13.24    |
|  2        |  0.8658   |  4.381    |
|  3        |  0.8353   |  9.721    |
|  4        |  0.8152   |  16.39    |
|  5        |  0.8708   |  6.907    |
|  6        |  0.8658   |  3.983    |
|  7        |  0.9493   |  1.45     |
|  8        |  0.8632   |  9.404    |
|  9        |  0.8632   |  5.774    |
|  10       |  0.8467   |  7.753    |
|  11       |  0.8037   |  20.81    |
|  12       |  0.8632   |  5.755    |
|  13       |  0.9076   |  2.624    |
|  14       |  0.8354   |  14.39    |
|  15       |  0.8442   |  13.42    |
|  16       |  0.8632   |  6.485    |
|  17       |  0.8353   |  10.32    |
|  18       |  0.9076   |  3.367    |
|  19       |  0.8823   |  2.479    |
|  20       |  0.8113   |  19.02    |
|  21       |  0.9493   |  1.102    |
|  22       |  0.8708   |  6.906    |
|  23       |  0.9493   |  1.277    |
|  24       |  0.8304   |  11.74    |
|  25       

In [23]:
best = knn_bo.max['params']
best['n_neighbors'] = round(best['n_neighbors'])


knn = KNeighborsClassifier(**best)
knn.fit(ba_tr, y_resampled)
pred = knn.predict(ba_te)
print(roc_auc_score(y_test, pred))

print(best)

0.866710875331565
{'n_neighbors': 1}


# RF

In [25]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", RandomForestClassifier(random_state = seed, n_jobs = -1))
                            ])

param_grid = {"clf__n_estimators": [50, 100, 150, 200, 250, 300],
              'clf__max_depth' : ['None', 5, ],
              'clf__min_samples_split': [2, 4, 6],
              'clf__min_samples_leaf': [1, 2, 3]

              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))


Fitting 10 folds for each of 108 candidates, totalling 1080 fits
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=50; total time=   1.6s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=50; total time=   1.5s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=50; total time=   1.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=50; total time=   1.1s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=50; total time=   1.2s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=50; total time=   1.1s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, clf__n_estimators=50; total time=   1.1s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=2, 

In [26]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", RandomForestClassifier(random_state = seed, n_jobs = -1))
                            ])

param_grid = {"clf__n_estimators": [50, 100, 150, 200, 250, 300],
              'clf__max_depth' : ['None', 5, ],
              'clf__min_samples_split': [2, 4, 6],
              'clf__min_samples_leaf': [1, 2, 3]
              }



gs_pipeline = RandomizedSearchCV(pipeline,param_distributions=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=4, clf__n_estimators=200; total time=   0.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=4, clf__n_estimators=200; total time=   1.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=4, clf__n_estimators=200; total time=   1.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=4, clf__n_estimators=200; total time=   1.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=4, clf__n_estimators=200; total time=   1.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=4, clf__n_estimators=200; total time=   1.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_split=4, clf__n_estimators=200; total time=   1.3s
[CV] END clf__max_depth=None, clf__min_samples_leaf=1, clf__min_samples_spli

In [28]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

def rf_bo_cl(n_estimators, max_depth, min_samples_split,min_samples_leaf):
    params_rf = {}
    params_rf['n_estimators'] = round(n_estimators)
    params_rf['max_depth'] = round(max_depth)
    params_rf['min_samples_split'] = round(min_samples_split)
    params_rf['min_samples_leaf'] = round(min_samples_leaf)
    
    scores = cross_val_score( RandomForestClassifier( random_state = seed, n_jobs = -1,**params_rf),
                             ba_tr, y_resampled, scoring=make_scorer(roc_auc_score), cv=10).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_rf ={
    'n_estimators':(50, 300),
    'max_depth': (5, 10), 
    'min_samples_split': (2, 6), 
    'min_samples_leaf': (1, 3),
 }
rf_bo = BayesianOptimization(rf_bo_cl, params_rf, random_state=111)
rf_bo.maximize(init_points=20, n_iter=50)
print('It takes %s minutes' % ((time.time() - start)/60))


best = rf_bo.max['params']
best['n_estimators'] = round(best['n_estimators'])
best['max_depth'] = round(best['max_depth'])
best['min_samples_split'] = round(best['min_samples_split'])
best['min_samples_leaf'] = round(best['min_samples_leaf'])



rf = RandomForestClassifier(random_state = seed, n_jobs = -1, **best)
rf.fit(ba_tr, y_resampled)
pred = rf.predict(ba_te)
print(roc_auc_score(y_test, pred))

print(best)

|   iter    |  target   | max_depth | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9276   |  8.061    |  1.338    |  3.744    |  242.3    |
|  2        |  0.8984   |  6.477    |  1.298    |  2.09     |  155.1    |
|  3        |  0.9035   |  6.193    |  1.675    |  5.963    |  109.4    |
|  4        |  0.8781   |  5.406    |  2.339    |  4.485    |  118.6    |
|  5        |  0.9148   |  7.331    |  1.237    |  2.296    |  275.2    |
|  6        |  0.9237   |  8.97     |  2.681    |  5.261    |  297.7    |
|  7        |  0.9175   |  7.886    |  2.628    |  3.685    |  56.86    |
|  8        |  0.9174   |  7.271    |  1.211    |  5.269    |  224.4    |
|  9        |  0.9161   |  7.826    |  1.548    |  5.994    |  84.51    |
|  10       |  0.9212   |  8.077    |  1.97     |  3.62     |  232.0    |
|  11       |  0.9187   |  6.614    |  1.801    |  3.276    |  286.8    |
|  12       |  0.9275   |  9.594    | 

# LGB

In [14]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", LGBMClassifier(random_state = seed, n_jobs = -1,boost_from_average = False))
                            ])

param_grid = {"clf__n_estimators": [50, 100, 150, 200, 250, 300],
              'clf__max_depth' : ['None', 5, ],
              'clf__learning_rate': [0.1, 0.01, 0.001,0.0001],
              'clf__min_child_samples': [20, 30, 40, 50]

              }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))


Fitting 10 folds for each of 192 candidates, totalling 1920 fits
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, clf__n_estimators=50; total time=   0.2s
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, clf__n_estimators=50; total time=   0.3s
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, clf__n_estimators=50; total time=   0.4s
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, clf__n_estimators=50; total time=   0.3s
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, clf__n_estimators=50; total time=   0.3s
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, clf__n_estimators=50; total time=   0.2s
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, clf__n_estimators=50; total time=   0.3s
[CV] END clf__learning_rate=0.1, clf__max_depth=None, clf__min_child_samples=20, 

In [15]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", LGBMClassifier(random_state = seed, n_jobs = -1,boost_from_average = False))
                            ])

param_grid = {"clf__n_estimators": [50, 100, 150, 200, 250, 300],
              'clf__max_depth' : ['None', 5, ],
              'clf__learning_rate': [0.1, 0.01, 0.001,0.0001],
              'clf__min_child_samples': [20, 30, 40, 50]
              }



gs_pipeline = RandomizedSearchCV(pipeline,param_distributions=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf__n_estimators=250; total time=   0.5s
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf__n_estimators=250; total time=   0.5s
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf__n_estimators=250; total time=   0.5s
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf__n_estimators=250; total time=   0.5s
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf__n_estimators=250; total time=   0.5s
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf__n_estimators=250; total time=   0.4s
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf__n_estimators=250; total time=   0.6s
[CV] END clf__learning_rate=0.001, clf__max_depth=5, clf__min_child_samples=40, clf

In [26]:
from bayes_opt import BayesianOptimization
from sklearn.model_selection import cross_val_score

def lgb_bo_cl(n_estimators, max_depth, learning_rate,min_child_samples):
    params_lgb = {}
    params_lgb['n_estimators'] = round(n_estimators)
    params_lgb['max_depth'] = round(max_depth)
    params_lgb['learning_rate'] = learning_rate
    params_lgb['min_child_samples'] = round(min_child_samples)
    
    scores = cross_val_score( LGBMClassifier(random_state = seed, n_jobs = -1,boost_from_average = False,**params_lgb),
                             ba_tr, y_resampled, scoring=make_scorer(roc_auc_score), cv=10).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_lgb ={
    'n_estimators':(50, 300),
    'max_depth': (5, 10), 
    'learning_rate': (0.0001,0.1), 
    'min_child_samples': (20, 50)
 }
lgb_bo = BayesianOptimization(lgb_bo_cl, params_lgb, random_state=111)
lgb_bo.maximize(init_points=20, n_iter=50)
print('It takes %s minutes' % ((time.time() - start)/60))


best = lgb_bo.max['params']
best['n_estimators'] = round(best['n_estimators'])
best['max_depth'] = round(best['max_depth'])
best['min_child_samples'] = round(best['min_child_samples'])



lgb = LGBMClassifier(random_state = seed, n_jobs = -1,boost_from_average = False, **best)
lgb.fit(ba_tr, y_resampled)
pred = lgb.predict(ba_te)
print(roc_auc_score(y_test, pred))

print(best)

|   iter    |  target   | learni... | max_depth | min_ch... | n_esti... |
-------------------------------------------------------------------------
|  1        |  0.9289   |  0.06126  |  5.845    |  33.08    |  242.3    |
|  2        |  0.9135   |  0.0296   |  5.746    |  20.67    |  155.1    |
|  3        |  0.8757   |  0.02394  |  6.688    |  49.72    |  109.4    |
|  4        |  0.8466   |  0.008211 |  8.348    |  38.64    |  118.6    |
|  5        |  0.939    |  0.04668  |  5.592    |  22.22    |  275.2    |
|  6        |  0.939    |  0.07942  |  9.203    |  44.46    |  297.7    |
|  7        |  0.9073   |  0.05777  |  9.069    |  32.64    |  56.86    |
|  8        |  0.9287   |  0.04547  |  5.527    |  44.52    |  224.4    |
|  9        |  0.901    |  0.05657  |  6.371    |  49.95    |  84.51    |
|  10       |  0.9301   |  0.06158  |  7.425    |  32.15    |  232.0    |
|  11       |  0.9287   |  0.03235  |  7.003    |  29.57    |  286.8    |
|  12       |  0.9429   |  0.09188  | 

# SVM

In [38]:
sv = SVC()
print(sv.get_params().keys())

dict_keys(['C', 'break_ties', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])


In [55]:
from sklearn.svm import SVC

start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", SVC(random_state = seed))
                            ])

param_grid = {"clf__gamma":  [0.01, 0.1, 0.3, 0.6, 1],
              'clf__kernel': ['rbf'],
              'clf__C': [0.01, 0.1, 0.5, 1, 10, 12] }



gs_pipeline = GridSearchCV(pipeline, param_grid=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))


Fitting 10 folds for each of 30 candidates, totalling 300 fits
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.4s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.4s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END ......clf__C=0.01, clf__gamma=0.01, clf__kernel=rbf; total time=   0.3s
[CV] END .......clf__C=0.01, clf__gamma=0.1, clf__kernel=rbf; total time=   0.4s
[CV] END .......clf__C=0.01, clf__gamma=0.1, c

In [64]:
start = time.time()


pipeline = Pipeline(steps= [('PCA', PCA(n_components = 23)),
                            ("clf", SVC(random_state = seed))
                            ])

param_grid = {"clf__gamma":  uniform(0.01,0.1),
              'clf__kernel': ['rbf'],
              'clf__C': uniform(8,12)
              }



gs_pipeline = RandomizedSearchCV(pipeline,param_distributions=param_grid, verbose=2, scoring=make_scorer(roc_auc_score), cv = 10)
gs_pipeline.fit(x_resampled, y_resampled)

end = time.time()

print(f"{end - start:.5f} sec")

pred = gs_pipeline.predict(x_test)

print('best_params is :', gs_pipeline.best_params_ , 'best_train_auc:',gs_pipeline.best_score_,'best_test_auc :', roc_auc_score(y_test, pred))

Fitting 10 folds for each of 10 candidates, totalling 100 fits
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.3s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.3s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.4s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.4s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.3s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.4s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.3s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.3s
[CV] END clf__C=12.154859169429082, clf__gamma=0.06661128283326562, clf__kernel=rbf; total time=   0.4s
[

In [63]:


def sv_bo_cl(C, gamma):
    params_sv = {}
    params_sv['C'] = C
    params_sv['gamma'] = gamma
    
    scores = cross_val_score( SVC(kernel = 'rbf',random_state = seed,**params_sv),
                             ba_tr, y_resampled, scoring=make_scorer(roc_auc_score), cv=10).mean()
    score = scores.mean()
    return score
# Run Bayesian Optimization
start = time.time()
params_sv ={
    'C':(5, 15),
    'gamma': (0.01, 0.1)
 }
sv_bo = BayesianOptimization(sv_bo_cl, params_sv, random_state=111)
sv_bo.maximize(init_points=20, n_iter=50)
print('It takes %s minutes' % ((time.time() - start)/60))


best = sv_bo.max['params']
best['C'] = best['C']
best['gamma'] = best['gamma']



sv = SVC(kernel ='rbf',random_state = seed,**best)
sv.fit(ba_tr, y_resampled)
pred = sv.predict(ba_te)
print(roc_auc_score(y_test, pred))

print(best)

|   iter    |  target   |     C     |   gamma   |
-------------------------------------------------
|  1        |  0.9366   |  11.12    |  0.02522  |
|  2        |  0.8793   |  9.361    |  0.07923  |
|  3        |  0.9379   |  7.953    |  0.02342  |
|  4        |  0.9173   |  5.225    |  0.04782  |
|  5        |  0.9263   |  7.387    |  0.04039  |
|  6        |  0.9314   |  14.91    |  0.0314   |
|  7        |  0.8894   |  5.812    |  0.07026  |
|  8        |  0.9289   |  11.21    |  0.03468  |
|  9        |  0.9456   |  9.662    |  0.02065  |
|  10       |  0.8641   |  5.74     |  0.09107  |
|  11       |  0.8692   |  12.94    |  0.08565  |
|  12       |  0.8578   |  13.15    |  0.09919  |
|  13       |  0.8718   |  10.77    |  0.08324  |
|  14       |  0.948    |  9.213    |  0.01247  |
|  15       |  0.9481   |  9.541    |  0.01948  |
|  16       |  0.8881   |  13.17    |  0.0728   |
|  17       |  0.9289   |  10.65    |  0.03468  |
|  18       |  0.9405   |  14.98    |  0.02242  |


0.5
